## Pipeline de Processamento e Armazenamento de Embeddings para Documentos SEI UTFPR

Este pipeline foi projetado para **extrair, limpar e converter o conteúdo de arquivos HTML de publicações do SEI UTFPR** em **representações vetoriais (embeddings)**. Essas representações são então armazenadas em um banco vetorial para permitir consultas semânticas futuras.

---

### Ferramentas Principais

* **Processamento de HTML**: `BeautifulSoup` para parsear e extrair texto de documentos HTML.
* **Manipulação de texto**: Bibliotecas Python padrão (`re`, `os`, etc.) para limpeza e tratamento do texto extraído.
* **Geração de Embeddings**: [sentence-transformers](https://www.sbert.net/) para a criação dos vetores semânticos.
* **Armazenamento Vetorial**: [ChromaDB](https://www.trychroma.com/) para gerenciar e indexar os embeddings.

---

### Estratégia de Chunking e Metadata

Cada documento HTML é minuciosamente processado e dividido em **chunks de texto limpos e relevantes**. O processo de chunking segue as seguintes diretrizes:

* **Limites de Tamanho**: Chunks são criados com um limite mínimo de 100 caracteres e um máximo de 500 caracteres, buscando otimizar a coerência semântica e a eficiência da busca.
* **Remoção de Ruído**: Frases irrelevantes e repetitivas (como "entrará em vigor", "considerando", "A autenticidade deste documento" e "assinado eletronicamente") são ignoradas para garantir que apenas o conteúdo significativo seja embedded.

Cada chunk é enriquecido com **metadados extraídos diretamente do documento original**, o que facilita buscas e filtragens futuras. Esses metadados incluem:

* Tipo de documento
* Órgão emissor
* Data de publicação
* Título
* Link formatado para o documento original

---

### Modelos de Embeddings Utilizados

Para cada chunk, embeddings são gerados usando uma variedade de modelos da família SentenceTransformer, o que permite flexibilidade e a possibilidade de comparar o desempenho de diferentes representações:

* `all-MiniLM-L6-v2`
* `all-mpnet-base-v2`
* `paraphrase-multilingual-MiniLM-L12-v2`
* `distiluse-base-multilingual-cased-v2`
* `stsb-xlm-r-multilingual`
* `neuralmind/bert-base-portuguese-cased`

---

### Armazenamento no ChromaDB

Os **embeddings, os textos originais e seus metadados** são armazenados em **coleções separadas no ChromaDB**. Cada modelo de embedding possui sua própria coleção, o que oferece múltiplos benefícios:

* **Comparação de Performance**: Permite avaliar e comparar o desempenho de diferentes modelos de embedding em consultas posteriores.
* **Flexibilidade**: Garante a liberdade de escolher o modelo de embedding mais adequado para cada tipo de consulta.
* **Rastreabilidade**: Os metadados associados a cada chunk asseguram a rastreabilidade e a origem das informações.

---

### Fluxo do Pipeline

1.  **Extração**: Leitura e extração do texto dos arquivos HTML do SEI UTFPR.
2.  **Pré-processamento e Chunking**: Limpeza do texto extraído, remoção de ruídos e divisão em chunks semânticos e otimizados.
3.  **Embeddings**: Geração de vetores para cada chunk usando todos os modelos de embedding definidos.
4.  **Armazenamento**: Persistência dos embeddings, textos e metadados nas coleções específicas do ChromaDB.
5.  **Monitoramento**: Geração de logs detalhados para acompanhamento do processamento e tratamento de eventuais erros.

In [ ]:
# Célula 1: Importando bibliotecas

import os
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import chromadb
from chromadb.config import Settings
from typing import List, Dict, Tuple
import chardet
from pathlib import Path
import random
import json

In [ ]:
# Célula 2: Configuração dos Modelos de Embeddings
def setup_embedding_models() -> List[Tuple[str, SentenceTransformer]]:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    models = [
        ("all-MiniLM-L6-v2", SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)),
        ("all-mpnet-base-v2", SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)),
        ("paraphrase-multilingual-MiniLM-L12-v2", SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device=device)),
        ("distiluse-base-multilingual-cased-v2", SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2', device=device)),
        ("stsb-xlm-r-multilingual", SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual', device=device)),
        ("neuralmind-bert-base-portuguese-cased", SentenceTransformer('neuralmind/bert-base-portuguese-cased'))
    ]
    return models

In [3]:
# Célula 3: Configuração do chromaDB

def setup_chromadb_collections(base_dir: str, models: List[Tuple[str, SentenceTransformer]]):
    """
    Configura o cliente ChromaDB e cria coleções para cada modelo de embedding.
    Se a coleção já existir, ela será excluída e recriada.
    """
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    collections = []
    existing_collections = [col.name for col in chroma_client.list_collections()]

    for model_name, _ in models:
        collection_name = f"documentos-SEI__{model_name}"

        if collection_name in existing_collections:
            print(f"Excluindo coleção existente: {collection_name}")
            chroma_client.delete_collection(name=collection_name)

        print(f"Criando nova coleção: {collection_name}")
        collection = chroma_client.create_collection(
            name=collection_name,
            metadata={"description": f"Documentos SEI usando modelo {model_name}"}
        )
        collections.append(collection)

    return chroma_client, collections

In [4]:
# Célula 4: Processar arquivos HTML

def extract_metadata(soup, file_name):
    """
    Extrai metadados do documento HTML, como tipo de documento, órgão, data de publicação, título e link.
    O link é formatado seguindo o padrão do SEI UTFPR.
    """
    meta = {}
    title = soup.title.get_text(strip=True) if soup.title else ''
    m = re.search(r'-\s*(\d+)\s*-\s*(.+)', title)
    meta['tipo_documento'] = m.group(2).strip() if m else 'Documento'
    meta['órgão'] = 'UTFPR'
    stamp = soup.find(text=re.compile(r'Boletim de Serviço Eletrônico em'))
    if stamp:
        data_match = re.search(r'(\d{2}/\d{2}/\d{4})', stamp)
        meta['data_publicação'] = data_match.group(1) if data_match else ''
    else:
        meta['data_publicação'] = ''
    # O título agora é o conteúdo completo da tag <title>
    meta['título'] = title

    # Formatação do link do documento
    base_url = "https://sei.utfpr.edu.br/sei/publicacoes/"
    # Remove a extensão .html
    file_name = file_name.replace('.html', '')
    # Substitui o primeiro _ após .php por ?
    file_name = re.sub(r'\.php_', '.php?', file_name)
    # Monta o link completo
    meta['link_documento'] = base_url + file_name

    return meta

def paragraphs(soup):
    """
    Remove tags de ruído e extrai parágrafos limpos do HTML.
    """
    for tag in soup(['style', 'script', 'img']):
        tag.decompose()
    paras = []
    for p in soup.find_all('p'):
        txt = ' '.join(p.stripped_strings)
        txt = re.sub(r'\s+', ' ', txt)
        if txt and not txt.lower().startswith('boletim de serviço'):
            paras.append(txt)
    return paras

HEADER_RE = re.compile(
    r'^(?:\d+[\.\)]|[IVXLC]+\.)\s|^(CAP[IÍ]TULO|SEÇÃO|T[IÍ]TULO|Art\.|ART\.)'
)

def is_header(text):
    """
    Detecta se um texto é um cabeçalho baseado em regex e proporção de letras maiúsculas.
    """
    if HEADER_RE.search(text):
        return True
    upper_ratio = sum(1 for c in text if c.isupper()) / len(text) if len(text) > 0 else 0
    return len(text) < 80 and upper_ratio > 0.7

def build_chunks(paras, meta, min_chars=200, max_chars=1000):
    """
    Constrói chunks otimizados agrupando parágrafos por contexto semântico
    """
    chunks = []
    current_chunk = []
    current_length = 0
    ignored_phrases = ["entrará em vigor", "considerando",
                      "A autenticidade deste documento", "assinado eletronicamente"]

    # Primeiro agrupa parágrafos por seções
    sections = []
    current_section = []

    for p in paras:
        p = p.strip()
        if not p:
            continue

        # Detecta início de nova seção
        if is_header(p):
            if current_section:
                sections.append(current_section)
            current_section = [p]
        else:
            current_section.append(p)

    if current_section:
        sections.append(current_section)

    # Agora processa cada seção separadamente
    for section in sections:
        section_text = ' '.join(section)

        # Se a seção inteira for pequena, adiciona como um chunk
        if len(section_text) <= max_chars:
            if (len(section_text) >= min_chars and
                not any(phrase.lower() in section_text.lower() for phrase in ignored_phrases)):
                chunks.append({
                    'metadados': meta,
                    'texto': section_text
                })
            continue

        # Seção grande - divide em chunks mantendo contexto
        words = section_text.split()
        current_chunk_words = []
        current_word_count = 0
        target_word_count = 500  # ~1000 caracteres

        for word in words:
            current_chunk_words.append(word)
            current_word_count += 1

            if current_word_count >= target_word_count:
                chunk_text = ' '.join(current_chunk_words)
                if (len(chunk_text) >= min_chars and
                    not any(phrase.lower() in chunk_text.lower() for phrase in ignored_phrases)):
                    chunks.append({
                        'metadados': meta,
                        'texto': chunk_text
                    })
                current_chunk_words = []
                current_word_count = 0

        # Adiciona o último pedaço se necessário
        if current_chunk_words:
            chunk_text = ' '.join(current_chunk_words)
            if (len(chunk_text) >= min_chars and
                not any(phrase.lower() in chunk_text.lower() for phrase in ignored_phrases)):
                chunks.append({
                    'metadados': meta,
                    'texto': chunk_text
                })

    return chunks

def html_to_chunks(file_path):
    """
    Lê um arquivo HTML, extrai metadados e gera chunks de texto limpos.
    """
    raw = Path(file_path).read_bytes()
    enc = chardet.detect(raw)['encoding'] or 'utf-8'
    soup = BeautifulSoup(raw.decode(enc, errors='ignore'), 'html.parser')
    meta = extract_metadata(soup, Path(file_path).name)
    paras = paragraphs(soup)
    return build_chunks(paras, meta)

In [5]:
# Célula 5: Processar arquivos HTML e inserir no ChromaDB

def process_html_files_and_insert(html_files: List[str], models: List[Tuple[str, SentenceTransformer]], chroma_client, collections: List, min_char_count: int = 100, min_word_count: int = 5):
    """
    Processa uma lista de arquivos HTML, gera chunks e insere no ChromaDB para cada modelo.
    Ignora chunks com menos de min_char_count caracteres e menos de min_word_count palavras.
    """
    total_chunks_processed = {model_name: 0 for model_name, _ in models}
    errors = []

    for file_path in tqdm(html_files, desc="Processando arquivos HTML"):
        try:
            chunks = html_to_chunks(file_path)

            for chunk in chunks:
                if len(chunk['texto']) >= min_char_count and len(chunk['texto'].split()) >= min_word_count:
                    for model_name, model in models:
                        collection_name = f"documentos-SEI__{model_name}"
                        collection = next((c for c in collections if c.name == collection_name), None)
                        if not collection:
                            print(f"Coleção não encontrada: {collection_name}")
                            continue

                        try:
                            collection.add(
                                documents=[chunk['texto']],
                                metadatas=[chunk['metadados']],
                                ids=[f"{Path(file_path).stem}__{model_name}_{total_chunks_processed[model_name]}"],
                            )
                            total_chunks_processed[model_name] += 1
                        except Exception as e:
                            errors.append(f"Erro ao adicionar chunk ao ChromaDB ({file_path}, {model_name}): {e}")
                else:
                    errors.append(f"Chunk muito curto ({file_path}): {chunk['texto'][:100]}...")

        except Exception as e:
            errors.append(f"Erro ao processar arquivo HTML ({file_path}): {e}")

    return total_chunks_processed, errors

In [6]:
# Célula 6: Processar arquivo aleatório para teste

def get_html_files_from_dir(html_dir: str) -> list:
    """
    Retorna uma lista de arquivos HTML encontrados no diretório informado.
    """
    html_files = []
    for root, dirs, files in os.walk(html_dir):
        for file in files:
            if file.lower().endswith(".html"):
                html_files.append(os.path.join(root, file))
    return html_files

def display_chunks_from_file(file_path: str):
    """
    Exibe os chunks gerados a partir de um arquivo HTML.
    """
    chunks = html_to_chunks(file_path)
    for i, c in enumerate(chunks, 1):
        print(f'Chunk {i}:\n\n[metadados]\n{json.dumps(c["metadados"], ensure_ascii=False, indent=2)}\n\n[texto]\n{c["texto"][:400]}...\n')
    return chunks

# Defina o diretório de teste explicitamente
basedir = os.getcwd()
sei_dir = os.path.normpath(os.path.join(basedir, "..", "Webscraping", "Files", "HTML", "SEI"))
html_files = get_html_files_from_dir(sei_dir)

# Escolha um arquivo aleatório para teste
random_file = random.choice(html_files)
chunks = display_chunks_from_file(random_file)

Chunk 1:

[metadados]
{
  "tipo_documento": "Grad.: Resolução (COGEP)",
  "órgão": "UTFPR",
  "data_publicação": "30/11/2021",
  "título": "SEI/UTFPR - 2418729 - Grad.: Resolução (COGEP)",
  "link_documento": "https://sei.utfpr.edu.br/sei/publicacoes/controlador_publicacoes.php?acao=publicacao_visualizar&id_documento=2653915&id_orgao_publicacao=0"
}

[texto]
Art. 1º Regulamentar o retorno às atividades pedagógicas presenciais, bem como a oferta das modalidades híbrida e remota, nos cursos de graduação, cursos técnicos e nos Centros Acadêmicos de Línguas Estrangeiras Modernas (CALEMs), da Universidade Tecnológica Federal do Paraná́ (UTFPR), durante o ano letivo de 2022....

Chunk 2:

[metadados]
{
  "tipo_documento": "Grad.: Resolução (COGEP)",
  "órgão": "UTFPR",
  "data_publicação": "30/11/2021",
  "título": "SEI/UTFPR - 2418729 - Grad.: Resolução (COGEP)",
  "link_documento": "https://sei.utfpr.edu.br/sei/publicacoes/controlador_publicacoes.php?acao=publicacao_visualizar&id_documento

C:\Users\joaov\AppData\Local\Temp\ipykernel_29812\2762023336.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  stamp = soup.find(text=re.compile(r'Boletim de Serviço Eletrônico em'))


In [ ]:
# Célula 7: Processar todos os arquivos HTML e inserir no ChromaDB com batch por arquivo

# Parâmetros de execução
basedir = os.getcwd()
sei_dir = os.path.normpath(os.path.join(basedir, "..", "Webscraping", "Files", "HTML", "SEI"))

# Setup modelos e coleções
models = setup_embedding_models()
chroma_client, collections = setup_chromadb_collections(basedir, models)

# Buscar arquivos HTML
html_files = []
for root, dirs, files in os.walk(sei_dir):
    for file in files:
        if file.lower().endswith(".html"):
            html_files.append(os.path.join(root, file))
print(f"Total de arquivos HTML encontrados: {len(html_files)}")

# Executar processamento
min_char_count = 100
min_word_count = 5
total_chunks_processed = {model_name: 0 for model_name, _ in models}
errors = []

for file_path in tqdm(html_files, desc="Processando arquivos HTML"):
    try:
        chunks = html_to_chunks(file_path)
        print(f"  Arquivo: {Path(file_path).name}, Total de chunks gerados: {len(chunks)}")

        # Filtra chunks válidos para processamento em batch
        valid_chunks = [c for c in chunks if len(c['texto']) >= min_char_count and len(c['texto'].split()) >= min_word_count]
        texts = [c['texto'] for c in valid_chunks]
        metadatas = [c['metadados'] for c in valid_chunks]

        for model_idx, (model_name, model) in enumerate(models):
            collection = collections[model_idx]
            try:
                # Gera embeddings em batch para todos os chunks do arquivo
                embeddings = model.encode(texts, batch_size=32, show_progress_bar=False)
                ids = [f"{Path(file_path).stem}__{model_name}_{i}" for i in range(len(texts))]

                # Adiciona todos os chunks em batch na coleção
                collection.add(
                    embeddings=embeddings.tolist(),
                    documents=texts,
                    metadatas=metadatas,
                    ids=ids
                )
                total_chunks_processed[model_name] += len(texts)
                print(f"    Modelo: {model_name}, {len(texts)} chunks processados em batch")
            except Exception as e:
                errors.append(f"Erro embedding batch ({file_path}, {model_name}): {str(e)}")

    except Exception as e:
        errors.append(f"Erro processamento ({file_path}): {str(e)}")

# Relatório final
print("\n=== Relatório Final ===")
print(f"Total de arquivos processados: {len(html_files)}")
for model_name, _ in models:
    print(f"Total de chunks gerados ({model_name}): {total_chunks_processed[model_name]}")
print(f"Total de erros: {len(errors)}")
if errors:
    print("\nErros encontrados:")
    for error in errors[:10]:  # Mostra apenas os 10 primeiros erros
        print(f"- {error}")
    if len(errors) > 10:
        print(f"... (mais {len(errors)-10} erros)")

for collection in collections:
    print(f"\nTotal de documentos na collection {collection.name}: {collection.count()}")

In [12]:
print("=== Resumo das coleções no ChromaDB ===")
for collection in collections:
    total_docs = collection.count()
    print(f"- Coleção: {collection.name}")
    print(f"  Total de documentos (chunks): {total_docs}")

=== Resumo das coleções no ChromaDB ===
- Coleção: documentos-SEI__all-MiniLM-L6-v2
  Total de documentos (chunks): 10184
- Coleção: documentos-SEI__all-mpnet-base-v2
  Total de documentos (chunks): 10184
- Coleção: documentos-SEI__paraphrase-multilingual-MiniLM-L12-v2
  Total de documentos (chunks): 10184
- Coleção: documentos-SEI__distiluse-base-multilingual-cased-v2
  Total de documentos (chunks): 10184
- Coleção: documentos-SEI__stsb-xlm-r-multilingual
  Total de documentos (chunks): 10184
- Coleção: documentos-SEI__neuralmind-bert-base-portuguese-cased
  Total de documentos (chunks): 10184


In [14]:
# exibe todos os chunks de um arquivo aleatório
random_file = random.choice(html_files)
chunks = display_chunks_from_file(random_file)


Chunk 1:

[metadados]
{
  "tipo_documento": "Edital",
  "órgão": "UTFPR",
  "data_publicação": "31/10/2023",
  "título": "SEI/UTFPR - 3805921 - Edital",
  "link_documento": "https://sei.utfpr.edu.br/sei/publicacoes/controlador_publicacoes.php?acao=publicacao_visualizar&id_documento=4167412&id_orgao_publicacao=0"
}

[texto]
DIRETORIA DE PESQUISA E PÓS-GRADUAÇÃO - CAMPUS CURITIBA EDITAL DIRPPG-CT Nº 03/2024​ - PROGRAMA DE APOIO À PUBLICAÇÕES DE ARTIGOS DE ALTO IMPACTO A Diretoria de Pesquisa e Pós-Graduação (DIRPPG) do Campus Curitiba da Universidade Tecnológica Federal do Paraná (UTFPR), torna público que no período de 06/11/2023 a 15/03/2024 estarão abertas as inscrições para solicitação de recurso financeiro para A...

Chunk 2:

[metadados]
{
  "tipo_documento": "Edital",
  "órgão": "UTFPR",
  "data_publicação": "31/10/2023",
  "título": "SEI/UTFPR - 3805921 - Edital",
  "link_documento": "https://sei.utfpr.edu.br/sei/publicacoes/controlador_publicacoes.php?acao=publicacao_visualizar&

C:\Users\joaov\AppData\Local\Temp\ipykernel_29812\2762023336.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  stamp = soup.find(text=re.compile(r'Boletim de Serviço Eletrônico em'))
